In [1]:
import pandas as pd
import numpy as np
from sklearn.metrics import mean_squared_error
from CustomKNN import Hypothesis1
from CustomKNN import Hypothesis2
from CustomKNN import CustomKNN
from CustomKNN import Hypothesis1Version2

In [2]:
a = np.load('./preprocessed/collaborative_input/train_STC.npy')
b = np.load('./preprocessed/collaborative_input/test_STC.npy')
train_sample = pd.read_pickle('./preprocessed/collaborative_input/new_train_sample.pkl')
test_sample = pd.read_pickle('./preprocessed/collaborative_input/new_test_sample.pkl')
train_sample['STC'] = a
test_sample['STC'] = b
from sklearn.model_selection import train_test_split

val_sample, test_sample = train_test_split(test_sample, test_size=0.5, random_state=42)


In [3]:
import math
#custom grid search for CustomKNN classes
def grid_search(model, params_list):
    lowest_mse = math.inf
    lowest_params = []
    if(model is CustomKNN):
        m = model(train_sample)
    else:
        m = model(train_sample, 'STC')
        
    m.train()
    for tup in params_list:
        val_sample['predictions'] = val_sample[['reviewerID','movieID']].apply(m.test_real, args=tup, axis=1)
        val_loss = mean_squared_error(val_sample['overall'],val_sample['predictions'])
        if(val_loss < lowest_mse):
            lowest_mse = val_loss
            lowest_params = tup
    return lowest_params

In [4]:
grid_search(CustomKNN, [(5,),(10,),(20,),(40,)])

(40,)

In [5]:
grid_search(Hypothesis1, [(40, 0.1),(40,0.15),(40,0.20),(40,0.25)])

(40, 0.15)

In [6]:
grid_search(Hypothesis2, [(40, 0.1),(40,0.15),(40,0.20),(40,0.25)])

(40, 0.25)

In [7]:
grid_search(Hypothesis1Version2, [(40, 0.1),(40,0.15),(40,0.20),(40,0.25)])

(40, 0.1)

In [9]:
knn = CustomKNN(train_sample)
knn.train()
test_sample['predictions'] = test_sample[['reviewerID','movieID']].apply(knn.test_real, args=(40,), axis=1)
mean_squared_error(test_sample['overall'],test_sample['predictions'])

1.4590015388498903

In [10]:
knn = Hypothesis1(train_sample, 'STC')
knn.train()
test_sample['predictions'] = test_sample[['reviewerID','movieID']].apply(knn.test_real, args=(40,0.15), axis=1)
mean_squared_error(test_sample['overall'],test_sample['predictions'])

1.4485823536463902

In [11]:
knn = Hypothesis2(train_sample, 'STC')
knn.train()
test_sample['predictions'] = test_sample[['reviewerID','movieID']].apply(knn.test_real, args=(40,0.25), axis=1)
mean_squared_error(test_sample['overall'],test_sample['predictions'])

1.4338662239857467

In [12]:
knn = Hypothesis1Version2(train_sample, 'STC')
knn.train()
test_sample['predictions'] = test_sample[['reviewerID','movieID']].apply(knn.test_real, args=(40,0.1), axis=1)
mean_squared_error(test_sample['overall'],test_sample['predictions'])

1.464892020066456